In [3]:
import numpy as np
import pandas as pd

In [16]:

def generate_psm_data(n=1000, random_seed=42):
    np.random.seed(random_seed)
    
    # Covariates
    age = np.random.normal(loc=40, scale=10, size=n)
    income = np.random.normal(loc=50000, scale=10000, size=n)
    gender = np.random.choice(['M', 'F'], size=n)
    
    # Let's define a logistic function for treatment assignment
    # Probability of treatment increases with age and income, different for genders
    logit = -10 + 0.05 * age + 0.0001 * income + np.where(gender == 'F', 1, 0)
    p_treat = 1 / (1 + np.exp(-logit))
    treatment = np.random.binomial(n=1, p=p_treat)
    
    # Outcome depends on treatment plus covariates
    # Let's embed a treatment effect
    outcome = 200 + 0.5 * age + 0.01 * income + np.where(gender == 'F', 5, 0) \
              + treatment * 20 + np.random.normal(0, 10, size=n)
    
    df_psm = pd.DataFrame({
        'age': age,
        'income': income,
        'gender': gender,
        'treatment': treatment,
        'outcome': outcome
    })
    
    return df_psm

In [17]:
# Example usage:
df_psm = generate_psm_data()
df_psm.head()

,age,income,gender,treatment,outcome
0,44.967142,63993.554366,F,0,863.695924
1,38.617357,59246.336829,M,0,816.145179
2,46.476885,50596.303699,F,0,736.053235
3,55.230299,43530.632223,M,0,667.175920
4,37.658466,56982.233136,M,0,790.874463


In [18]:
from sklearn.preprocessing import LabelEncoder

df = df_psm.copy()

label_encoder = LabelEncoder()
df['gender_encoded'] = label_encoder.fit_transform(df['gender'])  # F=0 or 1, M=0 or 1 depending on alpha order
df.head()

,age,income,gender,treatment,outcome,gender_encoded
0,44.967142,63993.554366,F,0,863.695924,0
1,38.617357,59246.336829,M,0,816.145179,1
2,46.476885,50596.303699,F,0,736.053235,0
3,55.230299,43530.632223,M,0,667.175920,1
4,37.658466,56982.233136,M,0,790.874463,1


In [19]:
from sklearn.linear_model import LogisticRegression

# Define features (covariates)
X = df[['age', 'income', 'gender_encoded']]
y = df['treatment']

# Initialize and train logistic regression
log_reg = LogisticRegression(solver='lbfgs')
log_reg.fit(X, y)

# Predicted propensity scores
propensity_scores = log_reg.predict_proba(X)[:, 1]
df['propensity_score'] = propensity_scores

df[['age','income','gender','treatment','outcome','propensity_score']].head()

,age,income,gender,treatment,outcome,propensity_score
0,44.967142,63993.554366,F,0,863.695924,0.348332
1,38.617357,59246.336829,M,0,816.145179,0.126559
2,46.476885,50596.303699,F,0,736.053235,0.128969
3,55.230299,43530.632223,M,0,667.175920,0.056221
4,37.658466,56982.233136,M,0,790.874463,0.099827


In [10]:
# 4. Calculate propensity scores, using a logistic regression model
logistic = LogisticRegression(random_state=42)
logistic.fit(X, y)
propensity_scores = logistic.predict_proba(X)[:, 1]

# Add propensity scores to the dataframe
df_psm['propensity_score'] = propensity_scores

df_psm.head()

,age,income,gender,treatment,outcome,propensity_score
0,44.967142,63993.554366,F,0,863.695924,0.345092
1,38.617357,59246.336829,M,0,816.145179,0.126600
2,46.476885,50596.303699,F,0,736.053235,0.129450
3,55.230299,43530.632223,M,0,667.175920,0.056943
4,37.658466,56982.233136,M,0,790.874463,0.100207


In [20]:
# 4. Perform matching
# 4.1 Split into Treated and Control Groups
df_treated = df[df['treatment'] == 1].copy()
df_control = df[df['treatment'] == 0].copy()

df_treated['match_id'] = df_treated.index
df_control['match_id'] = df_control.index

def nearest_neighbor_match(df_treated, df_control):
    """
    For each treated observation, find the control observation
    with the closest propensity score.
    
    Returns a merged DataFrame with matched pairs.
    """

    # Sort both DataFrames by propensity score for efficient searching
    df_treated_sorted = df_treated.sort_values(by='propensity_score').reset_index(drop=True)
    df_control_sorted = df_control.sort_values(by='propensity_score').reset_index(drop=True)

    # We will store results in a list of matched pairs
    matched_pairs = []
    
    # We'll keep track of which control indices have been used already
    used_control_indices = set()
    
    # For each treated, find the nearest control
    for i, row_t in df_treated_sorted.iterrows():
        t_score = row_t['propensity_score']
        
        # Among the control not yet used, find the one with the smallest absolute distance
        df_control_unused = df_control_sorted[~df_control_sorted.index.isin(used_control_indices)]
        
        # Compute absolute distance
        distances = (df_control_unused['propensity_score'] - t_score).abs()
        min_idx = distances.idxmin()  # Index of control with smallest distance
        
        # Store the pair
        matched_pairs.append((row_t['match_id'], df_control_sorted.loc[min_idx, 'match_id']))
        
        # Mark this control as used
        used_control_indices.add(min_idx)
    
    # Create a DataFrame with matched pairs
    matched_df = pd.DataFrame(matched_pairs, columns=['treated_id', 'control_id'])
    return matched_df

In [21]:
# 4.3 Perform the matching
matched_pairs = nearest_neighbor_match(df_treated, df_control)
matched_pairs.head()

,treated_id,control_id
0,666,635
1,876,566
2,988,149
3,464,834
4,922,933


In [22]:
# 4.4 Merge matched pairs with treatment and control data
df_treated_matched = matched_pairs.merge(
    df_treated, left_on='treated_id', right_on='match_id', suffixes=('_pair','_treat')
)
df_matched = df_treated_matched.merge(
    df_control, left_on='control_id', right_on='match_id', suffixes=('_treat','_control')
)

df_matched.head()

,treated_id,control_id,age_treat,income_treat,gender_treat,treatment_treat,outcome_treat,gender_encoded_treat,propensity_score_treat,match_id_treat,age_control,income_control,gender_control,treatment_control,outcome_control,gender_encoded_control,propensity_score_control,match_id_control
0,666,635,37.450228,40006.977677,M,1,635.076312,1,0.019572,666,23.451433,45963.515368,M,0,657.131121,1,0.019895,635
1,876,566,34.823887,44378.319734,M,1,676.735704,1,0.027021,876,19.618755,50669.907172,M,0,718.766064,1,0.027027,566
2,988,149,37.790358,43585.184048,M,1,698.237777,1,0.028192,988,42.969847,41530.386517,M,0,652.694609,1,0.028434,149
3,464,834,38.876720,44194.765502,M,1,685.119500,1,0.031259,464,39.653151,43875.626254,M,0,647.071301,1,0.031265,834
4,922,933,41.329697,39272.569743,F,1,632.314082,0,0.036892,922,18.011940,48940.516457,F,0,703.434014,0,0.036974,933


In [23]:
# 5. Calculate treatment effect
df_matched['outcome_diff'] = df_matched['outcome_treat'] - df_matched['outcome_control']
ATT = df_matched['outcome_diff'].mean()
print(f"Estimated ATT (via nearest-neighbor matching): {ATT:.2f}")

Estimated ATT (via nearest-neighbor matching): 28.00


In [24]:
# 5.2 Confidence Interval
def bootstrap_ci(data, n_bootstrap=1000, alpha=0.05):
    """
    Simple bootstrap for a 2-sided CI on the mean difference.
    """
    n = len(data)
    bootstrap_means = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(data, size=n, replace=True)
        bootstrap_means.append(np.mean(sample))
    lower = np.percentile(bootstrap_means, 100*alpha/2)
    upper = np.percentile(bootstrap_means, 100*(1-alpha/2))
    return np.mean(bootstrap_means), (lower, upper)

mean_diff, (lower_ci, upper_ci) = bootstrap_ci(df_matched['outcome_diff'].values)
print(f"Bootstrapped ATT = {mean_diff:.2f} [{lower_ci:.2f}, {upper_ci:.2f}]")

Bootstrapped ATT = 28.01 [17.60, 39.16]


In [25]:
# 6. Assess Covariate Balance
matched_treated = df_matched[['age_treat','income_treat','gender_encoded_treat']]
matched_control = df_matched[['age_control','income_control','gender_encoded_control']]

print("Mean of covariates in matched treated group:")
print(matched_treated.mean())

print("\nMean of covariates in matched control group:")
print(matched_control.mean())

Mean of covariates in matched treated group:
age_treat                  43.050138
income_treat            58246.766688
gender_encoded_treat        0.426087
dtype: float64

Mean of covariates in matched control group:
age_control                  43.256378
income_control            57509.689807
gender_encoded_control        0.356522
dtype: float64
